# An analysis of potential theme park locations for the metropolitan Melbourne Area

This notebook contains the capstone project work done for the IBM Professional Certificate on Data Science. Complete details of this project are available in this [report](https://github.com/andremun/Coursera_Capstone/blob/master/report/report.pdf).

## Introduction

Melbourne, Victoria is the second largest metropolitan area in Australia, with an estimated 5.191 million inhabitants as of 2019. According to [Mercer's Quality of living city ranking](https://mobilityexchange.mercer.com/Insights/quality-of-living-rankings) it is of the cities with the highest quality of life in the world, with one of the best education systems in the country according to [Quarely Simmonds Best Student's cities](https://www.topuniversities.com/city-rankings/2018#sorting=rank+custom=rank+order=desc+search=), making it an attractive city for families. However, Melbourne lacks of large theme and attraction parks. According to the state government of Victoria, [the only two parks](https://liveinmelbourne.vic.gov.au/discover/things-to-do-in-melbourne/childrens-activities-in-melbourne) in the region are: 
	
- Luna Park located in the inner city suburb of St. Kilda, at 8 kilometres from the central business district, with an area of over 11 thousand square metres. It opened in 1912, contains 20 attractions and operates year round.
-  Adventure Park located in the nearby city of Geelong, at 92 kilometres from the central business district, with an area of over 2 square kilometres. It opened in 1994, contains 20 attractions and operates from October to April, which correspond to the summer months in Australia.
	
Therefore, **the aim of this project is to identify the location of a new attraction park** closer to the central business district than Adventure Park, and an available area for expansion larger than Luna Park. The location should be within the metropolitan region, close to population growth corridors, with a high number of families nearby. Ideally, the location should be close to a suburban train station and other amenities like shopping centres, museums or zoological parks. The project would not focus on the economical feasibility of the park, but the families in the vicinity should have a moderate income that would allow them visit the park often. Interested parties would be developers, government entities, and families.

This notebook is divided in the following sections: (1) Importing libraries, where all the necessary packages are loaded into memory, (2) Data collection and cleaning, where the data from the existing datasets are pre-processed, (3) ...


## 1 - Importing Libraries

For this work, standard libraries such as `pandas`, `numpy`, `sklearn` and `matplotlib` will be used.

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.distance import geodesic
import requests # library to handle requests
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
import folium # map rendering library
import os.path
print('Libraries imported.')

Libraries imported.


## 2 - Data collection and pre-processing

### Postcode data

The first dataset corresponds to the **postcodes** for the State of Victoria, as provided by [Zen10](https://zen10.com.au/melbourne-suburb-list/), a Search Engine Optimisation Consultancy based on Melbourne. The data is in the file `victoria_postcodes.csv` and was imported using the `pandas` package. It is noteworthy that some suburbs have more postcodes associated to it. Therefore, the data was pre-processed to group each postcode with a unique suburb name. Moreover, any suburb that does not correspond to the Melbourne metropolitan area (which are marked with the regions `vic far country` and `vic country`) were removed.

In [3]:
file_suburb_pcode = 'victoria_postcodes.csv'
data_suburb_pcode = pd.read_csv(file_suburb_pcode, names=['Postcode','Suburb','Region'])
data_suburb_pcode['Suburb'] = data_suburb_pcode['Suburb'].str.lower()
data_suburb_pcode['Region'] = data_suburb_pcode['Region'].str.lower()
data_suburb_pcode = data_suburb_pcode.loc[data_suburb_pcode['Region']!='vic far country']
data_suburb_pcode = data_suburb_pcode.loc[data_suburb_pcode['Region']!='vic country']
suburb_pcodes = pd.DataFrame(columns=['Suburb','Postcodes'])
suburb_names = data_suburb_pcode['Suburb'].value_counts().index

# For every single unique suburb
for ii in range(0,len(suburb_names)):
    # Extract the data for that unique postcode
    idx = data_suburb_pcode['Suburb'] == suburb_names[ii]
    pcs = data_suburb_pcode.loc[idx]['Postcode'].values.tolist()
    suburb_pcodes.loc[ii] = [suburb_names[ii],pcs]
    
suburb_pcodes.head(5)

,Suburb,Postcodes
0,melbourne,"[3000, 3001, 3004]"
1,dandenong south,"[3164, 3175]"
2,cranbourne west,[3977]
3,exford,[3338]
4,templestowe,[3106]


### Geographical data

The second dataset contains the **geographical** information for each suburb in the Melbourne metropolitan area. This data was originally in [GeoJSON format](https://github.com/stephenmuss/suburb-boundaries-geojson/blob/master/vic.json), and is more thoroughly documented in the [report](https://github.com/andremun/Coursera_Capstone/blob/master/report/report.pdf). This file was a bit tricky to pre-process in Jupyter Notebooks, due to a nested structure in the `geometry` variable, which contained the coordinates of the boundary points. Given the limtied amount of time for this project, I decided to use [MATLAB](https://www.mathworks.com/) to obtain the centroid and the area in squared kilometers. The interested reader can check the MATLAB script provided [here](https://github.com/andremun/Coursera_Capstone/blob/master/preprocess_vic_suburbs.m), whereas the figure below shows the boundary data.

![Processed boundaries of the Victorian suburbs. Image obtained through MATLAB](./report/victoria_suburb_map.png)

The result from this pre-processing is the `greater_melbourne_suburb_data.csv` file which is loaded using `pandas`, and joined with the postcode data.

In [4]:
file_suburb_coord = 'greater_melbourne_suburb_data.csv'
data_suburb_coord = pd.read_csv(file_suburb_coord)
data_suburb_coord['Suburb'] = data_suburb_coord['Suburb'].str.lower()
data_suburb = data_suburb_coord
data_suburb = data_suburb.join(suburb_pcodes.set_index('Suburb'), on='Suburb')
data_suburb.head(5)

,Suburb,Longitude,Latitude,Area,Postcodes
0,abbotsford,144.999797,-37.804704,2.223370,[3067]
1,aberfeldie,144.897425,-37.759636,1.971027,[3040]
2,airport west,144.881337,-37.723862,4.653908,[3042]
3,albanvale,144.768545,-37.746106,2.416946,[3021]
4,albert park,144.963193,-37.844753,4.345716,[3206]


For our purpose, it is also important to know the distance of each suburb to Melbourne's Central Business District (CBD). The first step is to use the `geopy` package to determine the coordinates of the CBD. Next, these coordinates will be compared to the coordinates of the suburb centroids.

In [5]:
melbourne_cbd_address = 'Melbourne, VIC'

geolocator = Nominatim(user_agent = "explorer")
melbourne_cbd_geoloc = geolocator.geocode(melbourne_cbd_address)
latitude = melbourne_cbd_geoloc.latitude
longitude = melbourne_cbd_geoloc.longitude
melb_cbd_coords = (latitude, longitude)
print('The geograpical coordinates of Melbourne are {}, {}.'.format(latitude, longitude))
data_suburb['Distance to CBD'] = pd.Series(dtype='float64')

for ii in range(0,data_suburb.shape[0]):
    suburb_coords = (data_suburb['Latitude'].iloc[ii], data_suburb['Longitude'].iloc[ii])
    data_suburb.loc[ii,'Distance to CBD'] = geodesic(melb_cbd_coords, suburb_coords).km
    
data_suburb.head()

The geograpical coordinates of Melbourne are -37.8142176, 144.9631608.


,Suburb,Longitude,Latitude,Area,Postcodes,Distance to CBD
0,abbotsford,144.999797,-37.804704,2.223370,[3067],3.394597
1,aberfeldie,144.897425,-37.759636,1.971027,[3040],8.380310
2,airport west,144.881337,-37.723862,4.653908,[3042],12.351101
3,albanvale,144.768545,-37.746106,2.416946,[3021],18.737313
4,albert park,144.963193,-37.844753,4.345716,[3206],3.389219


### Demographic data

The third and fourth datasets contain **demograpic** information for each postcode in the Melborune metropolitan area. This data was extracted from the [2016 Australian Census Data](https://www.abs.gov.au/websitedbs/D3310114.nsf/Home/Assuring%20Census%20Data%20Quality), using the [TableBuilder Application](https://www.abs.gov.au/websitedbs/censushome.nsf/home/tablebuilder). The third dataset corresponds to the population by age (in five year increments) and it is in the `age_by_postcode.csv` file, while the fourth dataset corresponds to the population by  weekly income and it is in the `income_by_postcode.csv` file. From this data we are interested in calculating:

1. The population size of each suburb.
2. The population objective, which are children under the age of 15.
3. The density of each suburb, defined as the area in squared kilometres divided over the population.
4. The affluent population of each suburb, or thsoe with an income in or above the national median of 66,000 dollars per year, or about 1,300 dollars per week before taxes.
5. The ratio of affluent density of each suburb, defined by the ratio between the affluent population and the total population.

Note that we do not care about postcodes whose population is zero; hence, they are removed from the tables. These results will be appended to our `data_suburb` dataframe.

In [6]:
file_age_by_pcode = 'age_by_postcode.csv'
data_age_by_pcode = pd.read_csv(file_age_by_pcode)
data_age_by_pcode.set_index('Postcode', inplace = True)

file_income_by_pcode = 'income_by_postcode.csv'
data_income_by_pcode = pd.read_csv(file_income_by_pcode)
data_income_by_pcode.set_index('Postcode', inplace = True)

ages_by_suburb = pd.DataFrame(columns=data_age_by_pcode.columns)
incomes_by_suburb = pd.DataFrame(columns=data_income_by_pcode.columns)

for ii in data_suburb.index:
    pcs = data_suburb['Postcodes'].iloc[ii]
    
    idx = data_age_by_pcode.index == str(pcs[0])
    for jj in range(1,len(pcs)):
        aux = data_age_by_pcode.index == str(pcs[jj])
        for kk in range(0,len(idx)):
            idx[kk] = idx[kk] or aux[kk]

    ages_by_suburb.loc[ii] =  data_age_by_pcode.loc[idx].sum()
    
    idx = data_income_by_pcode.index == str(pcs[0])
    for jj in range(1,len(pcs)):
        aux = data_income_by_pcode.index == str(pcs[jj])
        for kk in range(0,len(idx)):
            idx[kk] = idx[kk] or aux[kk]

    incomes_by_suburb.loc[ii] =  data_income_by_pcode.loc[idx].sum()

ages_by_suburb.index = data_suburb['Suburb']
ages_by_suburb.rename(columns={'Total': 'Population'}, inplace = True)
ages_by_suburb['Population under 15'] = ages_by_suburb[['0-4 years','5-9 years','10-14 years']].sum(axis=1)
ages_by_suburb = ages_by_suburb[['Population','Population under 15']]

incomes_by_suburb.index = data_suburb['Suburb']
incomes_by_suburb.rename(columns={'Total': 'Population by Income'}, inplace = True)
incomes_by_suburb['Affluent Population'] = incomes_by_suburb[['1,250-1,499 (65,000-77,999)',
                                                              '1,500-1,749 (78,000-90,999)',
                                                              '1,750-1,999 (91,000-103,999)',
                                                              '2,000-2,999 (104,000-155,999)',
                                                              '3,000 or more (156,000 or more)']].sum(axis=1)
incomes_by_suburb = incomes_by_suburb[['Affluent Population']]

data_suburb = data_suburb.join(ages_by_suburb, on='Suburb')
data_suburb = data_suburb.join(incomes_by_suburb, on='Suburb')

data_suburb = data_suburb.loc[data_suburb['Population']!=0]
data_suburb.index = range(0,data_suburb.shape[0])

data_suburb['Density'] = data_suburb[['Population']].divide(other = data_suburb['Area'], axis=0)
data_suburb['Afluent Ratio'] = data_suburb[['Affluent Population']].divide(other = data_suburb['Population'], axis=0)

data_suburb.head(5)

,Suburb,Longitude,Latitude,Area,Postcodes,Distance to CBD,Population,Population under 15,Affluent Population,Density,Afluent Ratio
0,abbotsford,144.999797,-37.804704,2.223370,[3067],3.394597,8199,648.0,2815.0,3687.65,0.343335
1,aberfeldie,144.897425,-37.759636,1.971027,[3040],8.380310,25939,4281.0,6833.0,13160.1,0.263426
2,airport west,144.881337,-37.723862,4.653908,[3042],12.351101,15762,2687.0,3106.0,3386.83,0.197056
3,albanvale,144.768545,-37.746106,2.416946,[3021],18.737313,54190,9489.0,3454.0,22420.9,0.0637387
4,albert park,144.963193,-37.844753,4.345716,[3206],3.389219,10366,1805.0,3721.0,2385.34,0.358962


### Foursquare API setup

The final set of data will be obtained from the Foursquare API. There are a few things that are of our interest:

1. The location of the nearest bus, train or tram station.
2. The location of other ammenities such as shopping malls, museums, zoological parks.

The first step is to set up the API calls with the ID and SECRET, which have been stored in a separate file, not available in the repository for security reasons.

In [7]:
fsquare_secret_file = '../secret.csv'
fsquare_secret_data = pd.read_csv(fsquare_secret_file)
CLIENT_ID = fsquare_secret_data['CLIENT_ID'][0]
CLIENT_SECRET = fsquare_secret_data['CLIENT_SECRET'][0]
VERSION = '20180605'

Next, we define a function that will make the calls automatically and return the closest amenities.

In [8]:
def makeRequest(names, latitudes, longitudes, category):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url =   'https://api.foursquare.com/v2/venues/explore?' + \
                '&client_id={}'.format(CLIENT_ID) + \
                '&client_secret={}'.format(CLIENT_SECRET) + \
                '&v={}'.format(VERSION) + \
                '&ll={},{}'.format(lat,lng) + \
                '&categoryId={}'.format(category) + \
                '&radius=10000&limit=50&sortByDistance=1'
            
        # make the GET request
        res = requests.get(url).json()
        # print(res)
        results = res['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, 
                             lat, 
                             lng, 
                             v['venue']['name'], 
                             v['venue']['location']['lat'], 
                             v['venue']['location']['lng'],  
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                             'Suburb Latitude', 
                             'Suburb Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)


def getClosestVenues(venue_data, suburb_data):
    
    venues = makeRequest(names = suburb_data['Suburb'],
                         latitudes = suburb_data['Latitude'],
                         longitudes = suburb_data['Longitude'],
                         category = venue_data['Category'].str.cat(sep=','))
    
    for jj in range(0,venue_data.shape[0]):
        venue_type = venue_data['Type'].iloc[jj]
        venue_varname = 'Closest ' + venue_type
        distance_varname = 'Distance to ' + venue_type
        suburb_data[venue_varname] = pd.Series(dtype='object')
        suburb_data[distance_varname] = pd.Series(dtype='float64')

        venues_trim = venues.loc[venues['Venue Category']==venue_type]
        venues_trim.index = range(0,venues_trim.shape[0])

        for ii in range(0,suburb_data.shape[0]):
            idx = venues_trim['Suburb']==suburb_data['Suburb'].loc[ii]
            if ~np.any(idx):
                continue
            
            aux = venues_trim.loc[idx]
            aux.index = range(0,aux.shape[0])
            
            suburb_coords = (aux['Suburb Latitude'].iloc[0], aux['Suburb Longitude'].iloc[0])
            venue_coords = (aux['Venue Latitude'].iloc[0], aux['Venue Longitude'].iloc[0])

            suburb_data.loc[ii, venue_varname] = aux['Venue'].iloc[0]
            suburb_data.loc[ii, distance_varname] = geodesic(suburb_coords, venue_coords).km

    return(suburb_data)

### Venue data

Now, we apply these operations to obtain the closest venues.

In [9]:
landmark_codes = {'Type':['Shopping Mall',
                          'Museum',
                          'Zoo'],
                  'Category':['4bf58dd8d48988d1fd941735',
                              '4bf58dd8d48988d181941735',
                              '4bf58dd8d48988d17b941735']}
transportation_codes = {'Type':['Bus Station',
                                'Train Station',
                                'Tram Station'],
                        'Category':['4bf58dd8d48988d1fe931735',
                                    '4bf58dd8d48988d129951735',
                                    '52f2ab2ebcbc57f1066b8b51']}

landmark_data = pd.DataFrame(landmark_codes)
transportation_codes = pd.DataFrame(transportation_codes)

landmark_file = 'landmarks_by_suburb.csv'
transportation_file = 'transportation_by_suburb.csv'

if os.path.isfile(landmark_file):
    landmarks_by_suburb = pd.read_csv(landmark_file)
else:
    landmarks_by_suburb = getClosestVenues(landmark_data, data_suburb.loc[:,['Suburb','Longitude','Latitude']])
    landmarks_by_suburb.to_csv(landmark_file)

if os.path.isfile(transportation_file):
    transportation_by_suburb = pd.read_csv(transportation_file)
else:
    transportation_by_suburb = getClosestVenues(transportation_data, data_suburb.loc[:,['Suburb','Longitude','Latitude']])
    transportation_by_suburb.to_csv(transportation_file)

landmarks_by_suburb.set_index('Suburb', inplace=True)
landmarks_by_suburb.drop(columns=['Unnamed: 0','Longitude','Latitude'], inplace=True)

transportation_by_suburb.set_index('Suburb', inplace=True)
transportation_by_suburb.drop(columns=['Unnamed: 0','Longitude','Latitude'], inplace=True)

data_suburb = data_suburb.join(transportation_by_suburb, on='Suburb')
data_suburb = data_suburb.join(landmarks_by_suburb, on='Suburb')
data_suburb.head(10)

,Suburb,Longitude,Latitude,Area,Postcodes,Distance to CBD,Population,Population under 15,Affluent Population,Density,...,Closest Train Station,Distance to Train Station,Closest Tram Station,Distance to Tram Station,Closest Shopping Mall,Distance to Shopping Mall,Closest Museum,Distance to Museum,Closest Zoo,Distance to Zoo
0,abbotsford,144.999797,-37.804704,2.223370,[3067],3.394597,8199,648.0,2815.0,3687.65,...,Victoria Park Station,0.778365,Tram Stop 13 - Federation Square (3/3a/5/6/16/...,3.191072,The Hive Shopping Centre,0.659479,National Sports Museum,2.069114,NaN,NaN
1,aberfeldie,144.897425,-37.759636,1.971027,[3040],8.380310,25939,4281.0,6833.0,13160.1,...,Essendon Station,1.685383,Tram Stop D18,7.836533,Highpoint Shopping Centre,1.674582,Victorian Archives Centre,5.693425,Melbourne Zoo,5.589666
2,airport west,144.881337,-37.723862,4.653908,[3042],12.351101,15762,2687.0,3106.0,3386.83,...,Oak Park Station,3.603540,NaN,NaN,Westfield Airport West,1.149260,NaN,NaN,NaN,NaN
3,albanvale,144.768545,-37.746106,2.416946,[3021],18.737313,54190,9489.0,3454.0,22420.9,...,St Albans Station,2.748726,NaN,NaN,Brimbank Shopping Centre,0.888575,NaN,NaN,NaN,NaN
4,albert park,144.963193,-37.844753,4.345716,[3206],3.389219,10366,1805.0,3721.0,2385.34,...,Prahran Station,2.415208,Tram Stop 20 @ Park Street,1.686962,Albert Park Village,1.238901,National Herbarium,1.999613,NaN,NaN
5,albion,144.816046,-37.776356,3.245401,[3020],13.621757,44784,7741.0,3896.0,13799.2,...,Ardeer Station,1.445043,NaN,NaN,Sunshine Marketplace,1.306505,NaN,NaN,NaN,NaN
6,alphington,145.030875,-37.780379,3.549580,[3078],7.047976,11638,1809.0,3169.0,3278.7,...,Alphington Station,0.220825,Tram Stop 29,5.249642,Fairfield Village,1.143890,Melbourne Museum,5.809228,NaN,NaN
7,altona,144.813054,-37.860082,20.914229,[3018],14.160069,12685,2101.0,2757.0,606.525,...,Westona Station,0.573421,NaN,NaN,Central Square Shopping Centre,3.666678,Seaworks,8.289092,NaN,NaN
8,altona meadows,144.784470,-37.880983,13.368719,[3028],17.385667,29198,5152.0,4135.0,2184.05,...,Laverton Station,2.187284,NaN,NaN,Central Square Shopping Centre,1.167679,NaN,NaN,NaN,NaN
9,altona north,144.834670,-37.837965,17.778935,[3025],11.615312,12155,2255.0,1786.0,683.674,...,Altona Station,3.276154,Tram Stop D18,9.624819,Borrack Square,1.073244,Seaworks,6.972854,NaN,NaN


## 3 - Suburb identification and Mapping

We are going to make a points system to decide which suburbs are more promissing. 

1. Add Points 10 Points for a nearby train station, 5 points for a tram station and 3 points for a bus station.
2. Substract 10 Points for each nearby amenity.


In [10]:
data_suburb_test = data_suburb

data_suburb_test['Points'] = pd.Series(dtype='int64')

var_name = ['Distance to Train Station',
            'Distance to Tram Station',
            'Distance to Bus Station',
            'Distance to Shopping Mall',
            'Distance to Museum',
            'Distance to Zoo']
pnt_venue = [10,5,3,-10,-10,-10]

for ii in range(0,len(var_name)):
    idx = data_suburb_test.loc[:,var_name[ii]].isnull()==False
    if ii==0:
        data_suburb_test.loc[idx,'Points'] = pnt_venue[ii]
    else:
        data_suburb_test.loc[idx,'Points'] = data_suburb_test.loc[idx,'Points'] + pnt_venue[ii]
    
data_suburb_test = data_suburb_test.loc[data_suburb_test['Points']>=8]
data_suburb_test.sort_values(by='Density', ascending=True)

data_suburb_test = data_suburb_test.loc[data_suburb_test['Density']<=1000]
data_suburb_test.sort_values(by='Population under 15', ascending=False)

data_suburb_test = data_suburb_test.loc[data_suburb_test['Population under 15']>=500]
data_suburb_test.sort_values(by='Distance to CBD', ascending=True)

data_suburb_test = data_suburb_test.loc[data_suburb_test['Distance to CBD']<=60]
data_suburb_test.sort_values(by='Affluent Population', ascending=False)

,Suburb,Longitude,Latitude,Area,Postcodes,Distance to CBD,Population,Population under 15,Affluent Population,Density,...,Distance to Train Station,Closest Tram Station,Distance to Tram Station,Closest Shopping Mall,Distance to Shopping Mall,Closest Museum,Distance to Museum,Closest Zoo,Distance to Zoo,Points
371,parwan,144.444221,-37.772680,120.913830,[3340],45.939175,21123,4414.0,3492.0,174.695,...,9.445547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
22,avonsleigh,145.484745,-37.911650,9.289678,[3782],47.153901,7824,1476.0,1453.0,842.225,...,3.048362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
172,emerald,145.436472,-37.946061,63.375671,[3782],44.136234,7824,1476.0,1453.0,123.454,...,1.419608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
287,macclesfield,145.488903,-37.878529,36.512306,[3782],46.820644,7824,1476.0,1453.0,214.284,...,6.325835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
258,kilmore,144.927489,-37.306109,116.470642,[3764],56.482057,9161,1866.0,1367.0,78.655,...,5.157628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
259,kilmore east,145.011221,-37.278276,69.102919,[3764],59.634445,9161,1866.0,1367.0,132.57,...,2.988418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
119,cockatoo,145.512777,-37.929879,44.737715,[3781],50.032808,4471,1016.0,664.0,99.9381,...,3.815150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
316,monbulk,145.428493,-37.874150,24.356140,[3793],41.494197,3575,726.0,511.0,146.78,...,5.572706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0


Now we map these suburbs

In [11]:
# create map of Melbourne using latitude and longitude values
map_melbourne_metro = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, suburb in zip(data_suburb_test['Latitude'], data_suburb_test['Longitude'], data_suburb_test['Suburb']):
    label = '{}, VIC'.format(suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne_metro)  
    
map_melbourne_metro